In [2]:
import pandas as pd
import os

from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
)

from sqlalchemy import text
import sqlite3


In [40]:
df = pd.read_csv('data/df_lectures.csv')
df.columns
print(df[['Instructor', 'Instructor Name', 'Matched_Name']].head())
print(df['Instructor'].nunique())

         Instructor   Instructor Name      Matched_Name
0    Alekzandr Wray               NaN          No Match
1    Alekzandr Wray               NaN          No Match
2  Sonnet H. Retman  Sonnet H. Retman  Sonnet H. Retman
3  Sonnet H. Retman  Sonnet H. Retman  Sonnet H. Retman
4      Connie C. So      Connie C. So      Connie C. So
2600


In [4]:

df_renamed = df.rename(columns = {'prefix' : 'department_abbrv', 'GenEd Requirements' : 'credit_type', 'mean' : 'mean_gpa',
                                  'Course Code' : 'course_id', 'Credits' : 'credits', 'Term' : 'quarter_offered',
                                  'Rating' : 'teacher_rating', 'Start' : 'start_time', 'End' : 'end_time', 'Days' : 'meeting_days',
                                  'Instructor' : 'professor_name'})

df_final_small = df_renamed[['department_abbrv', 'course_id', 'credits', 'quarter_offered', 'mean_gpa', 'teacher_rating', 'start_time', 'end_time', 'meeting_days', 'professor_name', 'credit_type', 'UUID']]

In [5]:
df_final_small[df_final_small['department_abbrv'] == 'HSTRY']

,department_abbrv,course_id,credits,quarter_offered,mean_gpa,teacher_rating,start_time,end_time,meeting_days,professor_name,credit_type,UUID
1540,HSTRY,HSTRY 388,5,Spring 2025,3.602762,3.6,13:30,15:20,W,Susan A. Glenn,SSc,HSTRY388ASpring2025
1541,HSTRY,HSTRY 388,5,Spring 2025,3.602762,4.0,12:30,14:20,MW,Bruce Hevly,SSc,HSTRY388BSpring2025
1542,HSTRY,HSTRY 388,5,Spring 2025,3.602762,4.1,13:30,15:20,TTh,Arbella Bet-Shlimon,SSc,HSTRY388DSpring2025
1543,HSTRY,HSTRY 390,5,Spring 2025,0.000000,4.0,12:30,14:20,MW,Bruce Hevly,SSc,HSTRY390ASpring2025
1544,HSTRY,HSTRY 395,5,Spring 2025,3.927419,4.2,11:30,13:20,W,Raymond Jonas,SSc,HSTRY395ASpring2025
1545,HSTRY,HSTRY 494,5,Spring 2025,3.836923,4.0,15:30,17:20,T,Vicente L. Rafael,SSc,HSTRY494ASpring2025
1546,HSTRY,HSTRY 494,5,Spring 2025,3.836923,4.6,09:30,11:20,T,Elena Campbell,SSc,HSTRY494BSpring2025
1547,HSTRY,HSTRY 495,1-5,Spring 2025,4.000000,NaN,NaN,NaN,NaN,NaN,NaN,HSTRY495ASpring2025
1548,HSTRY,HSTRY 495,5,Spring 2025,4.000000,NaN,11:30,12:20,Th,Julie Osborn,NaN,HSTRY495BSpring2025
1549,HSTRY,HSTRY 498,5,Summer 2025,3.488925,3.8,09:40,11:50,F,Nathan E Roberts,SSc,HSTRY498ASummer2025


In [12]:
df_final_small['credit_type'].value_counts()

credit_type
A&H              553
NSc              475
SSc              430
C                111
SSc, DIV          94
DIV, SSc          90
SSc, A&H          68
A&H, SSc          50
RSN               41
RSN, NSc          37
NSc, RSN          36
SSc, NSc          27
NSc, SSc          25
A&H, DIV          21
RSN, SSc          15
SSc, A&H, DIV     13
DIV, A&H          13
DIV               11
A&H, NSc           8
SSc, RSN           8
SSc, DIV, A&H      5
A&H, SSc, DIV      4
NSc, A&H           4
DIV, A&H, SSc      4
DIV, SSc, A&H      4
A&H, DIV, SSc      4
SSc, RSN, NSc      4
DIV, SSc, NSc      3
NSc, RSN, SSc      2
DIV, NSc           2
A&H, RSN, SSc      2
NSc, SSc, DIV      2
SSc, DIV, NSc      2
A&H, RSN           2
RSN, NSc, SSc      1
Name: count, dtype: int64

In [6]:
df_final_small['course_id'] = df_final_small['course_id'].str.extract(r'(\d+)', expand=False).astype(int)

C:\Users\lucas\AppData\Local\Temp\ipykernel_26832\3533756964.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_small['course_id'] = df_final_small['course_id'].str.extract(r'(\d+)', expand=False).astype(int)


In [7]:
print(len(df_final_small))
print(df_final_small['UUID'].nunique())

6198
6198


In [8]:
df.columns

Index(['Unnamed: 0', 'Campus', 'Term', 'Course Code', 'Course Title',
       'Course Description', 'Activity Type', 'Section', 'Credits', 'Days',
       'Time', 'Building', 'Room', 'Instructor', 'Credit',
       'GenEd Requirements', 'Registration Code', 'enrollCount',
       'enrollMaximum', 'myPlanLink', 'index', 'mean', 'median', 'sd',
       'fourpointo_per', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21',
       '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33',
       '34', '35', '36', '37', '38', '39', '40', 'Start', 'End', 'prefix',
       'Matched_Name', 'Name', 'Rating', 'Difficulty', 'Link', 'Department',
       'Instructor Name', 'amount_learned_median', 'grading_techniques_median',
       'instructors_contribution_median', 'instructors_effectiveness_median',
       'instuctors_interest_median', 'the_course_as_a_whole_median',
       'the_course_content_median', 'Clean_Cour

In [9]:
conn = sqlite3.connect('databases/final_small.db')
cursor = conn.cursor()

# Define table name
table_name = 'all_classes'  # Adjust this to match your actual table name


# Insert DataFrame into the SQLite table
df_final_small.to_sql(table_name, conn, index=False)

6198

In [11]:
conn = sqlite3.connect('databases/final_large.db')
cursor = conn.cursor()

# Define table name
table_name = 'all_classes'  # Adjust this to match your actual table name


# # Insert DataFrame into the SQLite table
df.to_sql(table_name, conn, index=False)

6198

In [3]:
sql = """SELECT department_abbrv, course_id, uuid\nFROM all_classes\nWHERE professor_name LIKE 'Susan A. Glenn';

"""
engine = create_engine("sqlite:///databases/final_small.db")
with engine.connect() as con:
        rows = con.execute(text(sql))

  

In [ ]:
for row in rows:
    print(type(row[2]))